# Read/Write Text Files
- Use `open()` to read/write text files with proper modes and encoding.  
- Specify `encoding='utf-8'` for portability.  
- Leverage `with` to ensure files close automatically.  
- Read via iteration, `.read()`, `.readline()`, `.readlines()`.  
- Write via `.write()` or `.writelines()`, managing newlines manually.


| Method | Description |
|--------|-------------|
| read | Read from the underlying buffer (containing all the characters of the file) until we hit EOF and return string. |
| readline | Read until newline or EOF (first line as single string). |
| readlines | Returns a list of lines. |
| write | Write the given bytes to the underlying stream and return number of bytes written. Line separators are not added, so specify separators, newline as needed. Cannot write numerical values, convert numbers to strings before writing. |
| writelines | Write a list of lines to the stream. Line separators are not added, so specify separators, newline as needed. |


In [5]:
try:
    with open("files/config.txt", "w", encoding="utf-8") as file:
        file.write("Setting=Value\n") # add newline explicitly
        file.write("Other=Another\n") # add newline explicitly
        file.write(str(100))

    with open("files/config.txt", "r", encoding="utf-8") as file:
        content = file.read()
        print(f"Contents of file:")
        print(content)
except OSError as e:
    print(f"File error: {e}")

Contents of file:
Setting=Value
Other=Another
100


## File Modes
- `'r'`: read text (default), error if file missing.  
- `'w'`: write text, create or truncate.  
- `'a'`: append text, create if missing.  
- `'x'`: exclusive create, error if exists (good to prevent overwrites). Will raise `FileExistsError` if the file already exists.
- `'b'`: binary mode variant (e.g. `'rb'`, `'wb'`).  
- `'+'`: update mode, allows read/write (e.g. `'r+'`, `'w+'`).

### Understanding `+`

| Mode | Reads? | Writes? | Creates if missing? | Truncates on open? |
|------|--------|---------|---------------------|--------------------|
| r    | ✅      | ❌       | ❌                   | ❌                 |
| r+   | ✅      | ✅       | ❌                   | ❌                 |
| w    | ❌      | ✅       | ✅                   | ✅                 |
| w+   | ✅      | ✅       | ✅                   | ✅                 |
| a    | ❌      | ✅       | ✅                   | ❌                 |
| a+   | ✅      | ✅       | ✅                   | ❌                 |

In [13]:
from pathlib import Path

path = Path("./files/mode_demo.txt")

with path.open(mode="w", encoding="utf-8") as file:
    file.write("Initial line\n")

with path.open(mode="a", encoding="utf-8") as file:
    file.write("Appended line\n")

try:
    with path.open(mode="x", encoding="utf-8") as file:
        file.write("This will fail if file exists\n")
except FileExistsError as e:
    print(e)

[Errno 17] File exists: 'files/mode_demo.txt'


## Reading Text Files
- **Iteration:** `for line in f:`  
  - **When to use:** Ideal for processing large files **line by line** without loading the entire file into memory; lazy and very memory-efficient.

- **`f.read(size = -1)`**  
  - `size` can be used to specify the maximum number of characters to read; if negative or omitted, reads **the entire file**.  
  - **When to use:** When you need to grab a **chunk** of text (e.g. next 1024 chars). Good for bulk reads; but beware of high memory usage if you read the whole file at once.

- **`f.readline(size = -1)`**  
  - `size` can be used to specify the maximum number of characters to read from the line; if negative or omitted, reads **the full line** up to and including the newline.  
  - **When to use:** When you want **one line** at a time but need to guard against overly long lines. Returns an empty string when you reach EOF.

- **`f.readlines(hint = -1)`**  
  - `hint` can be used to define the approximate total number of bytes to read; if negative or omitted, reads **all lines**.  
  - **When to use:** When the file is small or moderate in size and you want a **list** of all lines for easy indexing or list comprehensions. Not recommended for very large files (may exhaust memory).

In [24]:
from pathlib import Path

sample = Path("files/read_demo.txt")
sample.write_text("First\nSecond\nThird\n", encoding="utf-8")

print("Iteration for reading:")
with sample.open(mode="r", encoding="utf-8") as file:
    for line in file:
        # There is linebreak from newline char in each line in the original file
        # and additionally, print() addds newline by default. So, use strip() to 
        # remove the newline char from the original text.
        print(f"\t-> {line.strip()}") 

        # Alternatively, set `end` char of `print`
        #print(f"\t-> {line}", end="")

print("read() for reading:")
with sample.open(mode="r", encoding="utf-8") as file:
    print(file.read()) 

print("readline() for reading:")
with sample.open(mode="r", encoding="utf-8") as file:
    print(file.readline())

print("readlines() for reading:")
with sample.open(mode="r", encoding="utf-8") as file:
    print(file.readlines())

Iteration for reading:
	-> First
	-> Second
	-> Third
read() for reading:
First
Second
Third

readline() for reading:
First

readlines() for reading:
['First\n', 'Second\n', 'Third\n']


## Writing Text Files
- **`f.write(s)`**  
  - `s` is the string to write; **does not** add a newline automatically.  
  - **When to use:** When writing **single strings** or building content piece by piece. Returns the number of characters written, so you can verify success.

- **`f.writelines(lines: Iterable[str]) -> None`**  
  - `lines` can be any iterable of strings; **does not** add newlines for you.
  - **When to use:** When you need to write a **batch** of strings at once (for example, a list of CSV rows). It's more efficient than multiple calls to `.write()`, but you must include `\n` at the end of each string if you want line breaks.

In [29]:
from pathlib import Path

write_demo = Path("files/write_demo.txt")

with write_demo.open(mode="w", encoding="utf-8") as file:
    file.write("Line A\n") # add newline explicitly
    file.write("Line B\n") # add newline explicitly

lines_to_write = [
    "user,ip,role",
    "alice,10.0.0.0,admin",
    "bob,10.0.0.1,dev",
    "charlie,10.0.02,audit"
]
with write_demo.open(mode="w", encoding="utf-8") as file:
    file.writelines(f"{line}\n" for line in lines_to_write) # add newline explictly